# 🎨 Trellis NIM - Simple Deployment & Inference

A streamlined notebook for deploying NVIDIA Trellis NIM and generating 3D models from text prompts using command line interface.

## What This Notebook Does:
1. **🔧 GPU Check** - Verify L40s GPU availability  
2. **🚀 Deploy NIM** - Run the Trellis NIM container
3. **⚡ Generate Models** - Command line 3D generation from text prompts

## Key Features:
- ✅ **Simple setup** - Just run the cells in order
- ✅ **Command line generation** - Direct API calls to NIM service
- ✅ **GLB output** - High-quality 3D models ready for Blender, Unity, etc.
- ✅ **Container management** - Easy start, stop, and status checking

**🚀 From setup to 3D models in minutes!**

---


## Step 1: Check GPU Availability

Verify that we have access to an L40s GPU:


In [ ]:
!nvidia-smi


## Step 2: Deploy Trellis NIM Container

**⚠️ IMPORTANT**: Replace `<PASTE_API_KEY_HERE>` with your actual NGC API key from https://ngc.nvidia.com/setup/api-key


In [ ]:
# Set your NGC API key here
import os
os.environ['NGC_API_KEY'] = '<PASTE_API_KEY_HERE>'  # Replace with your actual API key

# Verify the API key is set
if os.environ['NGC_API_KEY'] == '<PASTE_API_KEY_HERE>':
    print('❌ Please replace <PASTE_API_KEY_HERE> with your actual NGC API key!')
else:
    print('✅ NGC API key is set')


In [ ]:
# Docker login to NGC registry
!echo "$NGC_API_KEY" | docker login nvcr.io --username '$oauthtoken' --password-stdin


In [ ]:
# Create cache directory
!mkdir -p ~/.cache/nim
!chmod 777 ~/.cache/nim
!echo "Cache directory created at: ~/.cache/nim"


In [ ]:
# Start Trellis NIM container (this will run in the background)
# Note: This may take several minutes to download and start the first time

!docker run -d --name=nim-server \
  --runtime=nvidia --gpus='"device=0"' \
  -e NGC_API_KEY=$NGC_API_KEY \
  -p 8000:8000 \
  -v ~/.cache/nim:/opt/nim/.cache/ \
  nvcr.io/nim/microsoft/trellis:latest

print('🚀 Trellis NIM container started!')
print('⏳ Please wait 2-5 minutes for the container to fully initialize...')


In [ ]:
# Check container status
!docker ps | grep nim-server
!echo "\n📊 Container logs (last 10 lines):"
!docker logs nim-server --tail 10


In [ ]:
# 🌐 Launch Open Web UI - Interactive 3D Generation Interface
# 
# This launches a beautiful web interface to generate 3D models interactively!
# The UI connects to your deployed Trellis NIM following the NVIDIA Build platform pattern.

import os
import subprocess
import time
import webbrowser
from pathlib import Path

print("🚀 Starting Open Web UI for Trellis 3D Generation...")
print("🔗 Following NVIDIA Build pattern: https://build.nvidia.com/microsoft/trellis")

# Navigate to the web UI directory
web_ui_path = Path("./prompt-to-trellis")
if not web_ui_path.exists():
    print("❌ Web UI directory not found at ./prompt-to-trellis")
    print("💡 Make sure you're running this from the trellis-image-vision root directory")
else:
    print("✅ Found Web UI directory")
    
    # Check if NIM is running
    nim_check = subprocess.run(['docker', 'ps'], capture_output=True, text=True)
    if 'nim-server' in nim_check.stdout:
        print("✅ Trellis NIM is running")
    else:
        print("⚠️  Warning: Trellis NIM container not detected. Make sure it's running first!")
        print("💡 Run the deployment cells above to start the NIM container")
    
    # Change to web UI directory and install dependencies if needed
    os.chdir(web_ui_path)
    
    # Check if node_modules exists
    if not Path("node_modules").exists():
        print("📦 Installing dependencies...")
        subprocess.run(['npm', 'install'], check=True)
    
    print("🌐 Starting development server...")
    print("📍 Web UI will be available at: http://localhost:5173")
    print("🎯 This provides the same interface as https://build.nvidia.com/microsoft/trellis")
    print("✨ You can now prompt for 3D models in your browser!")
    
    # Start the development server in the background
    server_process = subprocess.Popen(['npm', 'run', 'dev'], 
                                    stdout=subprocess.PIPE, 
                                    stderr=subprocess.PIPE)
    
    # Wait a moment for server to start
    time.sleep(3)
    
    # Open browser
    webbrowser.open('http://localhost:5173')
    
    print("\n🎉 Open Web UI is now running!")
    print("📖 Usage:")
    print("  • Enter your text prompt in the interface")
    print("  • Click 'Generate 3D Model' to create your model")
    print("  • Download the .glb file when generation completes")
    print("\n⚠️  To stop the server: Press Ctrl+C in the terminal or restart this kernel")
    
    # Keep the process running - in Jupyter this will show output as it comes
    try:
        server_process.wait()
    except KeyboardInterrupt:
        print("\n🛑 Stopping Web UI server...")
        server_process.terminate()
        print("✅ Web UI server stopped")


## Step 3: Generate 3D Models (Command Line Method)

**🌐 RECOMMENDED: Use the Web Interface (Cell 14) for the best experience!**

Alternatively, you can generate models via command line by changing the prompt below and running the cell:


In [ ]:
#CHANGE THIS PROMPT TO GENERATE DIFFERENT 3D MODELS. BRANDS OR SPECIFIC PRODUCTS WILL FAIL THE GENERATION. 
PROMPT = "A simple coffee shop interior"

#Change output filename
OUTPUT_FILE = "result.glb"

# Optional: Change the seed for different variations (0 = random)
SEED = 0

# First, let's test if the NIM container is responding
print("🔍 Testing NIM container connectivity...")
health_check = !curl -s http://localhost:8000/health || curl -s http://localhost:8000/ || echo "CONNECTION_FAILED"

if "CONNECTION_FAILED" in ' '.join(health_check):
    print("❌ Cannot connect to NIM container at http://localhost:8000")
    print("💡 Make sure the container is running with: docker ps | grep nim-server")
    print("💡 Check container logs with: docker logs nim-server")
else:
    print("✅ NIM container is responding")

print("\n📡 Sending generation request to Trellis NIM...")

# Use the exact curl command format from your original request
import subprocess
import json
import base64

# Create the JSON payload
json_payload = json.dumps({
    "prompt": PROMPT,
    "seed": SEED
})

# Execute curl command exactly as specified
curl_cmd = [
    'curl', '-X', 'POST', 'http://localhost:8000/v1/infer',
    '-H', 'Accept: application/json',
    '-H', 'Content-Type: application/json',
    '-d', json_payload,
    '--silent'
]

print(f"🎯 Generating: '{PROMPT}'")
print("⏳ This may take 1-3 minutes...")

try:
    # Run the curl command
    result = subprocess.run(curl_cmd, capture_output=True, text=True, timeout=300)
    
    if result.returncode != 0:
        print(f"❌ Curl command failed with return code: {result.returncode}")
        print(f"Error: {result.stderr}")
        print(f"Command: {' '.join(curl_cmd)}")
    else:
        response_text = result.stdout.strip()
        print(f"📨 Raw response length: {len(response_text)} characters")
        print(f"📨 Response preview: {response_text[:300]}...")
        
        if not response_text:
            print("❌ Empty response from server")
        elif response_text.startswith('<!DOCTYPE') or '<html' in response_text:
            print("❌ Received HTML response instead of JSON - check if container is fully started")
        else:
            try:
                # Parse JSON response
                response_data = json.loads(response_text)
                print("✅ Valid JSON response received")
                
                # Check for artifacts in response
                if 'artifacts' in response_data and len(response_data['artifacts']) > 0:
                    artifact = response_data['artifacts'][0]
                    if 'base64' in artifact:
                        base64_data = artifact['base64']
                        
                        # Decode and save the GLB file
                        glb_data = base64.b64decode(base64_data)
                        
                        with open(OUTPUT_FILE, 'wb') as f:
                            f.write(glb_data)
                        
                        print(f"✅ SUCCESS! 3D model saved as: {OUTPUT_FILE}")
                        print(f"📊 File size: {len(glb_data) / 1024:.2f} KB")
                        print(f"🎯 You can now open {OUTPUT_FILE} in any 3D viewer or Blender!")
                    else:
                        print("❌ No base64 data in artifact")
                        print(f"Artifact keys: {list(artifact.keys())}")
                else:
                    print("❌ No artifacts found in response")
                    print(f"Response keys: {list(response_data.keys())}")
                    if 'error' in response_data:
                        print(f"Error from server: {response_data['error']}")
                        
            except json.JSONDecodeError as e:
                print(f"❌ Failed to parse JSON response: {e}")
                print(f"Raw response: {response_text}")

except subprocess.TimeoutExpired:
    print("❌ Request timed out (>5 minutes)")
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    import traceback
    traceback.print_exc()


## 🎉 That's it!

### To generate more 3D models:
1. Go back to the cell above with `PROMPT = "..."` and `OUTPUT_FILE = "... .glb"`
2. Change the prompt to whatever you want
3. Run both cells again

### Example prompts to try:
- `"A modern chair"`
- `"A futuristic car"`
- `"A wooden table"`
- `"A cozy bedroom"`
- `"A space station interior"`

### Useful commands:


In [ ]:
# Check if NIM container is running
!docker ps | grep nim-server


## Management Commands

Use these cells to manage your NIM container:


In [ ]:
# Check NIM container status
import subprocess

print("🔍 NIM STATUS CHECK")
print("="*40)

# Check NIM container
print("📦 NIM Container Status:")
nim_check = subprocess.run(['docker', 'ps'], capture_output=True, text=True)
if 'nim-server' in nim_check.stdout:
    print("✅ NIM container is running")
    !docker ps | grep nim-server
else:
    print("❌ NIM container is not running")

# Check NIM API health  
print("\n🔗 NIM API Health:")
import requests
try:
    response = requests.get("http://localhost:8000/health", timeout=5)
    if response.status_code == 200:
        print("✅ NIM API is responding")
    else:
        print(f"⚠️ NIM API returned status: {response.status_code}")
except:
    try:
        response = requests.get("http://localhost:8000/", timeout=5)
        print("✅ NIM service is responding")
    except:
        print("❌ NIM API is not accessible")

print("\n📊 Port 8000 Status:")
!netstat -an | grep ":8000" | grep LISTEN || echo "No service found on port 8000"


In [ ]:
# View recent container logs
!docker logs nim-server --tail 20


In [ ]:
# Stop the NIM container (when you're done)
!docker stop nim-server
!docker rm nim-server
print("🛑 Trellis NIM container stopped and removed")


## 🎉 Generation Complete!

### To generate more 3D models:
1. Go back to the cell above and change `PROMPT = "..."`
2. Optionally change `OUTPUT_FILE = "..."` to save with a different name
3. Run the cell again

### Example prompts to try:
- `"A modern chair"`
- `"A futuristic car"`
- `"A wooden table"`
- `"A cozy bedroom"`
- `"A space station interior"`
- `"A medieval castle tower"`
- `"A vintage coffee shop interior"`


In [ ]:
# View recent container logs
!docker logs nim-server --tail 20


In [ ]:
# Stop the NIM container (when you're done)
!docker stop nim-server
!docker rm nim-server
print("🛑 Trellis NIM container stopped and removed")


In [ ]:
# List generated GLB files
!ls -la *.glb


## 🎯 Complete!

### **🎉 Your Trellis NIM setup is now complete!**

**What you have:**
- ✅ **NVIDIA Trellis NIM** deployed and running
- ✅ **Command-line generation** - Simple, reliable 3D model creation
- ✅ **GLB output files** - Ready for Blender, Unity, or any 3D software
- ✅ **Container management** - Easy start, stop, and monitoring

### **🚀 Workflow:**
1. **Deploy NIM**: Run cells 2-8 (one-time setup)
2. **Generate models**: Use cell 10, change the prompt and run
3. **Manage container**: Use cells 12-15 for monitoring and cleanup

### **🛠️ Troubleshooting:**
- **Container not responding** → Check `docker logs nim-server`
- **Generation fails** → Verify container is running with `docker ps | grep nim-server`
- **Out of memory** → Restart container with `docker restart nim-server`

### **📁 Files:**
- **Generated Models**: `.glb` files in the current directory
- **NIM Service**: Running on http://localhost:8000

**🎨 Ready to create amazing 3D models from text prompts!**
